In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [4]:
from lib import data_management as dm

In [5]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import MXG, EXG

In [41]:
exa = Agilent_EXA_N9010A("EXA")
ro_awg = KeysightAWG("AWG3")
q_awg = KeysightAWG("AWG2")
ro_lo = EXG("EXG")

In [39]:
q_lo = MXG("MXG")

In [42]:
ro_lo = Agilent_PNA_L("PNA-L2")

In [26]:
reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [10]:
ro_resonator_frequency = 7.047e9
if_offset = 50e6

In [11]:
ro_lo.set_output_state = lambda x: x
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_span(0)

True

In [12]:
ro_lo.set_center(ro_resonator_frequency)

True

In [14]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "TEST", -20)
res = cal.calibrate(lo_frequency=ro_resonator_frequency, if_frequency=if_offset, lo_power=0,
                    ssb_power=-70, waveform_resolution=0.1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

Phase:  0.5534 [ -70.012 -102.440 -109.476 ]      115.817 ]       Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -20}
Radiation parameters: {'lo_frequency': 7047000000.0, 'lo_power': 0, 'if_frequency': 50000000.0, 'ssb_power': -70, 'waveform_resolution': 0.1}
Optimization results: {'dc': -106.69306945800781, 'if': [-70.01216125488281, -102.44039154052734, -109.4761962890625]}
Optimization parameters {'dc_offsets': array([ 0.06928289, -0.09399985]), 'if_offsets': array([ 0.06900207, -0.09600268]), 'if_amplitudes': array([-0.02473116,  0.04114605]), 'if_phase': array([ 1.73792908])}
Optimization time: 0 h 0 m 56.84 s
Finished at: 2017-04-24 12:09:37.619379


In [15]:
dm.save_IQMX_calibration(res)

In [7]:
calibrations=dm.load_IQMX_calibration_database("TEST", -20)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-30, lo_frequency=7e9, 
                                      if_frequency=10e6, waveform_resolution=0.1).items()))

## Qubit

In [31]:
qubit_frequency = 7.9e9
if_frequency = 50e6

In [44]:
cal = IQCalibrator(q_awg, exa, q_lo, "TESTQ", -6)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=15,
                    ssb_power=-20, waveform_resolution=.1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=100,
                    initial_guess=res.get_optimization_results()[0])
print(res)

Phase:  1.4804 [ -19.907 -67.896 -90.877 ]             ]          Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7950000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -20, 'waveform_resolution': 0.1}
Optimization results: {'dc': -81.5372314453125, 'if': [-19.906503677368164, -67.89619445800781, -90.87674713134766]}
Optimization parameters {'dc_offsets': array([ 0.04307401, -0.09063471]), 'if_offsets': array([ 0.04425181, -0.0921925 ]), 'if_amplitudes': array([ 0.28466617,  0.28035364]), 'if_phase': array([ 4.65071566])}
Optimization time: 0 h 1 m 4.93 s
Finished at: 2017-04-24 17:20:37.125641


In [45]:
dm.save_IQMX_calibration(res)

In [46]:
q_lo.visa_instr.close()

In [57]:
q_lo.set_power(15)

In [75]:
calibrations = (dm.load_IQMX_calibration_database("TESTQ", -20))
cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-40, lo_frequency=qubit_frequency+if_frequency, 
                                      if_frequency=if_frequency, waveform_resolution=0.1).items()))

In [76]:
print(cal)

Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -20}
Radiation parameters: {'lo_frequency': 9250000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -40, 'waveform_resolution': 0.1}
Optimization results: {'dc': -111.25095367431641, 'if': [-39.540977478027344, -83.05058288574219, -98.06412506103516]}
Optimization parameters {'dc_offsets': array([-0.4119494 ,  0.51782169]), 'if_offsets': array([-0.48734834,  0.68734449]), 'if_amplitudes': array([ 5.17142139,  5.58796915]), 'if_phase': array([ 1.6731673])}
Optimization time: 0 h 1 m 26.33 s
Finished at: 2017-03-27 21:10:28.338740


In [30]:
q_lo.visa_instr.close()